## Import Modules

In [ ]:
# Core
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from pyclustering.cluster.kmedoids import kmedoids


# Text Processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import wordcloud
from textblob import TextBlob
from nltk.corpus import stopwords
import nltk.collocations as nc
import spacy
nlp = spacy.load('en_core_web_sm')
# Remove stopwords, including google and tesla
stops = stopwords.words('english') + ['google', 'tesla', 'work']
en_stopwords = set(stops)
from textblob import TextBlob
from sklearn.decomposition import NMF, LatentDirichletAllocation

# Time Series
import seaborn as sns
sns.set(style="ticks", color_codes=True)

# Geo
import folium
import os

# Machine Learning
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from pyclustering.cluster.silhouette import silhouette_ksearch_type, silhouette_ksearch
import operator



## Import Datasets 
#### (Google & Tesla reviews web-scraped from Indeed)

In [ ]:
# Import datasets - web scraped data

# Code Source 
# https://stackoverflow.com/questions/16888888/how-to-read-a-xlsx-file-
# using-the-pandas-library-in-ipython

df_google = pd.read_excel('google_reviews.xlsx', sheet_name="sheet1")
df_tesla = pd.read_excel('tesla_reviews.xlsx', sheet_name="sheet1")

# Import state data for geo

# Code Source 
# https://developers.google.com/public-data/docs/canonical/states_csv
df_state_geo = pd.read_csv('states_geo.csv')

## EDA & Data Cleaning

In [ ]:
# Get some information about the datasets

# Code Source
# https://www.datacamp.com/community/tutorials/time-series-analysis-tutorial

df_google.info()
df_tesla.info()

# No missing values - all objects, one float64

####  Convert data types

In [ ]:
# Code Source
# https://www.datacamp.com/community/tutorials/time-series-analysis-tutorial

# Date to date data type
df_google.date = pd.to_datetime(df_google.date)
df_tesla.date = pd.to_datetime(df_tesla.date)

# Rating to integer
df_google.rating = df_google.rating.astype(np.int64)
df_tesla.rating = df_tesla.rating.astype(np.int64)

In [ ]:
# Take a look at the google dataframe
df_google.head(2)

In [ ]:
# Take a look at the tesla dataframe
df_tesla.head(2)


#### Create employee type (former or current employee) from reviewer variable

In [ ]:
# Strip out former/current employee and make it's own variable

# Code Source: 
# https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/

# split reviewer variable on '(' 
new_google = df_google["reviewer"].str.split("(", n = 1, expand = True) 
new_tesla = df_tesla["reviewer"].str.split("(", n = 1, expand = True) 

# make job title column from the left part of the split 
df_google["job_title"]= new_google[0] 
df_tesla["job_title"]= new_tesla[0] 
  
# make emp type column from the right part of the split
df_google["emp_status"]= new_google[1] 
df_tesla["emp_status"]= new_tesla[1] 

# Drop reviewer column 
df_google.drop('reviewer', inplace=True, axis=1)
df_tesla.drop('reviewer', inplace=True, axis=1) 

In [ ]:
# Now have 7 variables, created employee type & job title, dropped reviewer
df_google.shape

In [ ]:
df_tesla.shape

In [ ]:
# Get a count of each employee status category

# Code Source
# https://stackoverflow.com/questions/22391433/count-the-frequency
# -that-a-value-occurs-in-a-dataframe-column

df_google['emp_status'].value_counts()

# Lots of junk in there

In [ ]:
df_tesla['emp_status'].value_counts()

In [ ]:
# Pretty much an if statement - if emp_status contains 'Current', make it 'current', else make it 'former'

# Code Source
# https://stackoverflow.com/questions/18196203/
# how-to-conditionally-update-dataframe-column-in-pandas

df_google['emp_status'] = np.where(df_google['emp_status'].str.contains('Current'), 'current', 'former')

df_tesla['emp_status'] = np.where(df_tesla['emp_status'].str.contains('Current'), 'current', 'former')

In [ ]:
# Clean categories - current/former
df_google['emp_status'].value_counts()

In [ ]:
df_tesla['emp_status'].value_counts()

#### Create state from location variable

In [ ]:
# Strip out state and make it's own variable

# split location variable on ',' 
new_google = df_google["location"].str.split(",", n = 1, expand = True) 
new_tesla = df_tesla["location"].str.split(",", n = 1, expand = True) 

# make state column from the right part of the split 
df_google["state"]= new_google[1] 
df_tesla["state"]= new_tesla[1] 

# Drop location column 
df_google.drop('location', inplace=True, axis=1)
df_tesla.drop('location', inplace=True, axis=1) 

In [ ]:
# Take the first two characters

# Code Source
# https://stackoverflow.com/questions/36505847/substring-of-an-entire-column-in-pandas-dataframe

df_google.state = df_google.state.str.slice(1, 3)
df_tesla.state = df_tesla.state.str.slice(1, 3)

In [ ]:
# Upper case the state code extracted

# Code Source
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.upper.html

df_google.state = df_google.state.str.upper()
df_tesla.state = df_tesla.state.str.upper()

In [ ]:
# Get a count of each employee status category
df_google['state'].value_counts()

In [ ]:
# Validate states extracted against state list,
# Not found assign ERR

# Code Source
# https://thispointer.com/python-how-to-check-if-an-item-exists-in-list-search-by-value-or-condition/

def state_lookup(df):
    # If valid state code, keep it, else assign 'ERR'
    for i in range(0,len(df)):
        if df.iloc[i] in df_state_geo['state'].values.tolist():
            df[i] = df[i]
        else:
            df[i] = 'ERR'
        

In [ ]:
state_lookup(df_google.state)

In [ ]:
state_lookup(df_tesla.state)

In [ ]:
# Get a count of reviews in each state
df_google['state'].value_counts()

In [ ]:
# Get a count of reviews in each state
df_tesla['state'].value_counts()

In [ ]:
# Merge longitude/latitude to dataframe, may use later for geo

# Code source
# https://medium.com/importexcel/common-excel-task-
# in-python-vlookup-with-pandas-merge-c99d4e108988

df_google = pd.merge(df_google,df_state_geo, how='left', on='state')
df_tesla = pd.merge(df_tesla, df_state_geo, how='left', on='state')


In [ ]:
# Assign company variable
df_google['Company'] = 'google'
df_tesla['Company'] = 'tesla'

In [ ]:
# Union google and tesla datasets for bar chart

# Code Source
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

frames = [df_google, df_tesla]

df_comp = pd.concat(frames)

In [ ]:
# Change data type for categorical variables

# Code source
# https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html

df_comp['emp_status'] = df_comp['emp_status'].astype('category')
df_comp['Company'] = df_comp['Company'].astype('category')
df_comp['state'] = df_comp['state'].astype('category')

#### Explore newly created employee status variable

In [ ]:
# Barplot - Percentage of reviews by current/former empployees

# Code sournce
# https://stackoverflow.com/questions/35692781/python-plotting-percentage-in-seaborn-bar-plot

x, y, hue = "Company", "proportion", "emp_status"
# hue_order = ["Male", "Female"]

(df_comp[x]
 .groupby(df_comp[hue])
 .value_counts()
 .rename(y)
 .reset_index()
 .pipe((sns.barplot, "data"), x=x, y=y, hue=hue));

#### Explore rating variable

In [ ]:
# Look at Rating Summary for Google
df_google['rating'].describe()

In [ ]:
# Rating Frequency Bar Plot Google
df_google['rating'].value_counts().plot.bar(color='Gray');

In [ ]:
# Look at Rating Summary for Tesla
df_tesla['rating'].describe()

In [ ]:
# Rating Frequency Bar Plot Google
df_tesla['rating'].value_counts().plot.bar(color='Gray');

#### Explore state variable

In [ ]:
# State Frequency Bar Plot Google
df_google['state'].value_counts().plot.bar(color='Gray');

In [ ]:
# State Frequency Bar Plot Google
df_tesla['state'].value_counts().plot.bar(color='Gray');

#### TF-IDF


In [ ]:
# Combine title and reviews to clean for each review
# Trying to not combine -  many titles were duped in review
# google_review_text = df_google["title"] + ' ' + df_google["reviews"]
# tesla_review_text = df_tesla["title"] + ' ' + df_tesla["reviews"]

google_review_text = df_google["reviews"]
tesla_review_text = df_tesla["reviews"]

In [ ]:
# Take a closer look at combine review text
google_review_text[1]

In [ ]:
tesla_review_text[1]

In [ ]:
# Function to clean text

# Parts of code from Week 3 and Week 6 Solutions
# Regis Text Analytics Class - Dr. Nathan George

def preprocess_text(text_docs):
   
    # remove punctuation
    table = str.maketrans({key: None for key in string.punctuation})
    docs = [str(d).translate(table) for d in text_docs]
    # replace newlines with spaces
    docs = [re.sub('[\r\n]+', ' ', d) for d in docs]
    # replace website links with space -- want to do this before digits
    docs = [re.sub('https[\w]*', ' ', d) for d in docs]
    # replace digits with space
    docs = [re.sub('\d', ' ', d) for d in docs]
    # relpace multiple spaces with one
    docs = [re.sub('\s\s+', ' ', d) for d in docs]
    
    
    # process with spacy
    spacy_docs = [nlp(d) for d in docs]
    lemmatized_docs = []
    
    # keep the word if it's a pronoun, otherwise use the lemma
    lemmas = [[w.lemma_ if w.lemma_ != '-PRON-'
                   else w.lower_
                   for w in d if w.lower_ not in en_stopwords]
              for d in spacy_docs]
    
    
    pos = [[w.pos_ for w in d if w.lower_ not in en_stopwords] for d in spacy_docs]
    flat_lemmas = [i for l in lemmas for i in l]
    flat_pos = [i for p in pos for i in p]
    df = pd.DataFrame({'word': flat_lemmas, 'pos': flat_pos})
    
    lemmatized_docs = [' '.join(l) for l in lemmas]
    
    return lemmatized_docs, df

In [ ]:
# Run preprocess_text for reviews
cln_review_google, df_pos_google = preprocess_text(google_review_text)
cln_review_tesla, df_pos_tesla = preprocess_text(tesla_review_text)

In [ ]:
# Take a closer look at cleaned review
cln_review_google[1]

In [ ]:
cln_review_tesla[1]

In [ ]:
# Create lists of grouped words - part of speech, word

# Code Source
# From Text Analytics Week 3 - Dr. Nathan George

words_google = list(df_pos_google.groupby('word'))
words_tesla = list(df_pos_tesla.groupby('word'))

In [ ]:
# Gets value counts of words - word, pos, count

# From Text Analytics Week 3 - Dr. Nathan George

all_cnts_google = {}
for w, w_df in words_google:
    all_cnts_google[w] = w_df['pos'].value_counts()
    
all_cnts_tesla = {}
for w, w_df in words_tesla:
    all_cnts_tesla[w] = w_df['pos'].value_counts()

In [ ]:
# Total up nouns and verbs

# From Text Analytics Week 3 - Dr. Nathan George

nouns_google, verbs_google = {}, {}
for w in all_cnts_google:
    if 'NOUN' in all_cnts_google[w].keys():
        nouns_google[w] = all_cnts_google[w]['NOUN']
    
    if 'VERB' in all_cnts_google[w].keys():
        verbs_google[w] = all_cnts_google[w]['VERB']
        
        
nouns_tesla, verbs_tesla = {}, {}
for w in all_cnts_tesla:
    if 'NOUN' in all_cnts_tesla[w].keys():
        nouns_tesla[w] = all_cnts_tesla[w]['NOUN']
    
    if 'VERB' in all_cnts_tesla[w].keys():
        verbs_tesla[w] = all_cnts_tesla[w]['VERB']

In [ ]:
# Report out top 10 nouns and verbs per company

# Code Source from Week 3 Text Analytics - Dr. Nathan George
# https://stackoverflow.com/questions/613183/how-to-sort-a-dictionary-by-value

# Sorting most nouns/verbs to the least
top_nouns_google = sorted(nouns_google, key=nouns_google.get, reverse=True)
top_noun_counts_google = [nouns_google[n] for n in top_nouns_google]
top_verbs_google = sorted(verbs_google, key=verbs_google.get, reverse=True)
top_verb_counts_google = [verbs_google[v] for v in top_verbs_google]

top_nouns_tesla = sorted(nouns_tesla, key=nouns_tesla.get, reverse=True)
top_noun_counts_tesla = [nouns_tesla[n] for n in top_nouns_tesla]
top_verbs_tesla = sorted(verbs_tesla, key=verbs_tesla.get, reverse=True)
top_verb_counts_tesla = [verbs_tesla[v] for v in top_verbs_tesla]

print('top 10 Google nouns:\n')
for n, c in zip(top_nouns_google[:10], top_noun_counts_google[:10]):
    print(n, '(' + str(c) + ' times)')
    
print('\ntop 10 Google verbs:\n')
for n, c in zip(top_verbs_google[:10], top_verb_counts_google[:10]):
    print(n, '(' + str(c) + ' times)')
    
print('\ntop 10 Tesla nouns:\n')
for n, c in zip(top_nouns_tesla[:10], top_noun_counts_tesla[:10]):
    print(n, '(' + str(c) + ' times)')
    
print('\ntop 10 Tesla verbs:\n')
for n, c in zip(top_verbs_tesla[:10], top_verb_counts_tesla[:10]):
    print(n, '(' + str(c) + ' times)')

In [ ]:
# Get top n-grams from the reviews

# From Text Analytics Week 3 - Dr. Nathan George

def get_top_grams_google(docs, n=2, top=10):
    
    v_google = CountVectorizer(ngram_range=(n, n))
    grams_google = v_google.fit_transform(docs)
    # convert to array and flatten to avoid weird indexing
    gram_sum_google = np.array(np.sum(grams_google, axis=0)).flatten()
    gram_dict_google = {i: v for v, i in v_google.vocabulary_.items()}  # dictionary of index: word
    top_grams_google = gram_sum_google.argsort()[::-1]
    for i in top_grams_google[:top]:
        print('"' + gram_dict_google[i] + '" shows up', gram_sum_google[i], 'times')
    
    return [gram_dict_google[i] for i in top_grams_google], gram_sum_google[top_grams_google]

In [ ]:
# Report out top 10 for 1,2,3, and 4 grams - google

# From Text Analytics Week 3 - Dr. Nathan George

ngrams_google, ngram_counts_google = {}, {}
for n in [1, 2, 3, 4]:
    print('top 10', str(n) + '-grams:\n')
    ngrams_google[n], ngram_counts_google[n] = get_top_grams_google(cln_review_google, n=n)
    print('\n')

In [ ]:
def get_top_grams_tesla(docs, n=2, top=10):
    
    v_tesla = CountVectorizer(ngram_range=(n, n))
    grams_tesla = v_tesla.fit_transform(docs)
    # convert to array and flatten to avoid weird indexing
    gram_sum_tesla = np.array(np.sum(grams_tesla, axis=0)).flatten()
    gram_dict_tesla = {i: v for v, i in v_tesla.vocabulary_.items()}  # dictionary of index: word
    top_grams_tesla = gram_sum_tesla.argsort()[::-1]
    for i in top_grams_tesla[:top]:
        print('"' + gram_dict_tesla[i] + '" shows up', gram_sum_tesla[i], 'times')
    
    return [gram_dict_tesla[i] for i in top_grams_tesla], gram_sum_tesla[top_grams_tesla]

In [ ]:
# Report out top 10 for 1, 2,3, and 4 grams - tesla

ngrams_tesla, ngram_counts_tesla = {}, {}
for n in [1, 2, 3, 4]:
    print('top 10', str(n) + '-grams:\n')
    ngrams_tesla[n], ngram_counts_tesla[n] = get_top_grams_tesla(cln_review_tesla, n=n)
    print('\n')

In [ ]:
# TFIDF - fit/transform

# min_df=3 ignores terms occuring in less than 3 documents

#Code from Week 4 Solution - Regis Text Analytics Class - Dr. Nathan George

# Instantiate tfidf vectorizer and fit_transform
tfidf_vectorizer_google = TfidfVectorizer(min_df=3,ngram_range=(1,1))
train_tfidf_vectors_google = tfidf_vectorizer_google.fit_transform(cln_review_google)

tfidf_vectorizer_tesla = TfidfVectorizer(min_df=3,ngram_range=(1,1))
train_tfidf_vectors_tesla = tfidf_vectorizer_tesla.fit_transform(cln_review_tesla)

In [ ]:
train_tfidf_vectors_google.shape

In [ ]:
train_tfidf_vectors_tesla.shape

In [ ]:
# Instantiate wordcloud and set size
wc = wordcloud.WordCloud(width=800, height=400)

In [ ]:
# Wordcloud for 1,2,3,4 grams - google

# Week 3 Text Analytics - Dr. Nathan George

for n in [1, 2, 3, 4]:
    f = wc.generate_from_frequencies(frequencies={w: i for w, i in zip(ngrams_google[n], ngram_counts_google[n])})
    fig = plt.figure(figsize=(10, 10))
    _ = plt.imshow(f, interpolation='bilinear')
    _ = plt.axis("off")  # assign result to a dummy variable so it doesn't show anything
    _ = plt.title(str(n) + '-grams')

In [ ]:
# Wordcloud for 1,2,3,4 grams - tesla

for n in [1, 2, 3, 4]:
    f = wc.generate_from_frequencies(frequencies={w: i for w, i in zip(ngrams_tesla[n], ngram_counts_tesla[n])})
    fig = plt.figure(figsize=(10, 10))
    _ = plt.imshow(f, interpolation='bilinear')
    _ = plt.axis("off")  # assign result to a dummy variable so it doesn't show anything
    _ = plt.title(str(n) + '-grams')

## Time Series

In [ ]:
# Value counts by date

# Code Source
# https://stackoverflow.com/questions/22391433/
# count-the-frequency-that-a-value-occurs-in-a-dataframe-column

df_date_count_google = df_google['date'].value_counts()
df_date_count_tesla = df_tesla['date'].value_counts()


In [ ]:
# Save as a dataframe
df_date_google = pd.DataFrame(df_date_count_google)
df_date_tesla = pd.DataFrame(df_date_count_tesla)

In [ ]:
df_date_google.head()

In [ ]:
# Getting into format for time series plot
# reset and name index, rename columns

# Code Source
# https://stackoverflow.com/questions/26097916/
# convert-pandas-series-to-dataframe

df_count_google = df_date_google.date.to_frame().reset_index()
df_count_google = df_count_google.rename(columns= {'index': 'date', 'date': 'count' })
df_count_google.index.name = 'index'

df_count_tesla = df_date_tesla.date.to_frame().reset_index()
df_count_tesla = df_count_tesla.rename(columns= {'index': 'date', 'date': 'count' })
df_count_tesla.index.name = 'index'


In [ ]:
df_count_google.head(2)

In [ ]:
# Set date as the index for time series plots

# Code Source - # https://www.datacamp.com/community/tutorials/time-series-analysis-tutorial

df_count_google.set_index('date', inplace=True)
df_count_tesla.set_index('date', inplace=True)

In [ ]:
df_count_google.head(2)

In [ ]:
# Code Source
# https://stackoverflow.com/questions/38792122/how-to-group-and-count-rows-by-month-and-year-using-pandas

df_count_google = df_count_google.resample('MS').size()
df_count_tesla = df_count_tesla.resample('MS').size()

In [ ]:
# Plot time series google

# Code Source - 
# https://www.datacamp.com/community/tutorials/time-series-analysis-tutorial

df_count_google.plot(figsize=(20,10),linewidth = 5,fontsize=20)
plt.xlabel('Year',fontsize=20);


In [ ]:
# Plot time series tesla
df_count_tesla.plot(figsize=(20,10),linewidth = 5,fontsize=20)
plt.xlabel('Year',fontsize=20);

In [ ]:
# Plot time series rolling mean google

# Code Source - 
# https://www.datacamp.com/community/tutorials/time-series-analysis-tutorial

df_count_google.rolling(12).mean().plot(figsize=(20,10), linewidth=5, fontsize=20)
plt.xlabel('Year', fontsize=20);

In [ ]:
# Plot time series rolling mean tesla

df_count_tesla.rolling(12).mean().plot(figsize=(20,10), linewidth=5, fontsize=20)
plt.xlabel('Year', fontsize=20);

In [ ]:
# Look at day of week reviews were submitted

# Covert to day of the week (Monday=0, Sunday=6)

# Code Source
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.dayofweek.html

df_google_day_of_week = df_comp['date'].dt.dayofweek

In [ ]:
# Case statement to match integer to day of week text

# Code Source
# https://data-flair.training/blogs/python-switch-case/

def week(i):
    switcher={
        0:'Monday',
        1:'Tuesday',
        2:'Wednesday',
        3:'Thursday',
        4:'Friday',
        5:'Saturday',
        6:'Sunday'
             }
    return switcher.get(i,"Invalid day of week")

In [ ]:
# Creating file to plot day of week by company comparison

# Call function - add new variable to df_comp
df_comp['dayofweek'] = np.array([week(i) for i in df_google_day_of_week[0:]])

In [ ]:
# Change data type for categorical variables
df_comp['dayofweek'] = df_comp['dayofweek'].astype('category')

In [ ]:
# Barplot - review day of week by company

# Code Source
# https://stackoverflow.com/questions/35692781/python-plotting-percentage-in-seaborn-bar-plot

x, y, hue = "dayofweek", "proportion", "Company"

(df_comp[x]
 .groupby(df_comp[hue])
 .value_counts(normalize=True)
 .rename(y)
 .reset_index()
 .pipe((sns.barplot, "data",), x=x, y=y, hue=hue,));


## Sentiment Analysis

In [ ]:
# Grab reviews variable

# Code source:
# https://dev.to/rodolfoferro/sentiment-analysis-on-trumpss-tweets-using-python-

df_google_cln_rev = pd.DataFrame(data=cln_review_google, columns=['Reviews'])
df_tesla_cln_rev = pd.DataFrame(data=cln_review_tesla, columns=['Reviews'])

In [ ]:
# Using textblob for sentiment,
# assign 1, 0, -1 based on polarity

# Code source:
# https://dev.to/rodolfoferro/sentiment-analysis-on-trumpss-tweets-using-python-

def analize_sentiment(Reviews):
   
    analysis = TextBlob(Reviews)
    if analysis.sentiment.polarity >= .3:
        return 1
    elif analysis.sentiment.polarity < .3 and analysis.sentiment.polarity >= -.3 :
        return 0
    else:
        return -1

In [ ]:
# Call function and add SA to dataframe

# Code source
# https://dev.to/rodolfoferro/sentiment-analysis-on-trumpss-tweets-using-python-

df_google['SA'] = np.array([analize_sentiment(Reviews) for Reviews in df_google_cln_rev['Reviews']])
df_tesla['SA'] = np.array([analize_sentiment(Reviews) for Reviews in df_tesla_cln_rev['Reviews']])

In [ ]:
# Save out polarity to dataframe as well

def analize_polarity(Reviews):
   
    analysis = TextBlob(Reviews)
    return analysis.sentiment.polarity

In [ ]:
df_google['Polarity'] = np.array([analize_polarity(Reviews) for Reviews in df_google_cln_rev['Reviews']])
df_tesla['Polarity'] = np.array([analize_polarity(Reviews) for Reviews in df_tesla_cln_rev['Reviews']])

In [ ]:
df_google.head(2)

In [ ]:
df_tesla.head(2)

In [ ]:
# Look at sentiment compared to rating

# Code Source
# https://pbpython.com/pandas-crosstab.html

pd.crosstab(df_google.rating, df_google.SA)

In [ ]:
pd.crosstab(df_tesla.rating, df_tesla.SA)

In [ ]:
# Look at some reviews with positive sentiment and give a rating of 1
# https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas

df_tesla[['reviews','Polarity']].loc[(df_tesla['rating'] == 1) & (df_tesla['SA'] == 1)].head(5)

In [ ]:
# Check out how textblob is determing sentiment - idea from Dr. George
t = TextBlob('Same as every other tech company now. They will only pay the top people what they want then the rest are left to pick up the pieces. Tesla name carries weight and they know it.')

In [ ]:
# Tried to play with sentiment ranges above to improve accuracy, really need to train sentiment model
t.sentiment_assessments

## Geo

In [ ]:
# Create files with state/sentiment mean and state/rating mean

# Code Source
# https://towardsdatascience.com/pandas-tips-and-tricks-33bcc8a40bb9

df_google_state_SA = df_google.groupby('state')['SA'].mean()
df_tesla_state_SA = df_tesla.groupby('state')['SA'].mean()

df_google_state_rating = df_google.groupby('state')['rating'].mean()
df_tesla_state_rating = df_tesla.groupby('state')['rating'].mean()

In [ ]:
# Make them dataframes 

df_google_state_SA = pd.DataFrame(df_google_state_SA)
df_tesla_state_SA = pd.DataFrame(df_tesla_state_SA)

df_google_state_rating = pd.DataFrame(df_google_state_rating)
df_tesla_state_rating = pd.DataFrame(df_tesla_state_rating)

In [ ]:
# Reset indexes

df_google_state_SA.reset_index(inplace=True)
df_tesla_state_SA.reset_index(inplace=True)

df_google_state_rating.reset_index(inplace=True)
df_tesla_state_rating.reset_index(inplace=True)

In [ ]:
# Remove the ERR states

# https://stackoverflow.com/questions/34397982/pandas-dataframe-access-multiple-items-with-not-equal-to
notERR_google_SA = df_google_state_SA[df_google_state_SA['state'] != 'ERR']
notERR_tesla_SA = df_tesla_state_SA[df_tesla_state_SA['state'] != 'ERR']

# https://stackoverflow.com/questions/34397982/pandas-dataframe-access-multiple-items-with-not-equal-to
notERR_google_rating = df_google_state_rating[df_google_state_rating['state'] != 'ERR']
notERR_tesla_rating = df_tesla_state_rating[df_tesla_state_rating['state'] != 'ERR']

In [ ]:
# Looks good!
notERR_google_SA.head()

In [ ]:
# Folium choropleth of average sentiment by state - Google

# Code sources
# https://medium.com/@austinlasseter/using-folium-to-generate-a-simple-map-of-your-pandas-data-87ddc5d55f8d
# https://python-graph-gallery.com/292-choropleth-map-with-folium/
# https://github.com/bradtraversy/python_folium_example/edit/master/data/us-states.json

map = folium.Map(location=[48, -102], zoom_start=3)

#mac
state_geo = os.path.join('/Users/christiandavies/Desktop/data_pract_1/Project', 'us-states.json')

# windows 
# state_geo = os.path.join(r"C:\Users\Czdavies\Desktop\DS_prac_1\project", 'us-states.json')

map.choropleth(geo_data=state_geo, data=notERR_google_SA,
             columns=['state', 'SA'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Sentiment')

map


In [ ]:
# Folium choropleth of average rating by state - Google

map = folium.Map(location=[48, -102], zoom_start=3)

map.choropleth(geo_data=state_geo, data=notERR_google_rating,
             columns=['state', 'rating'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Rating')

map

In [ ]:
# Folium choropleth of average sentiment by state - Tesla


map = folium.Map(location=[48, -102], zoom_start=3)

map.choropleth(geo_data=state_geo, data=notERR_tesla_SA,
             columns=['state', 'SA'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Sentiment')

map


In [ ]:
# Folium choropleth of average rating by state - Tesla

map = folium.Map(location=[48, -102], zoom_start=3)

map.choropleth(geo_data=state_geo, data=notERR_tesla_rating,
             columns=['state', 'rating'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Rating')

map


## Unsupervised Learning - Clustering

In [ ]:
## Can't use - not getting accurate optimal # of clusters, just going to use TF-IDF

# Convert additional features for machine learning

# df_google_emp_status = pd.get_dummies(df_google['emp_status'])
# df_tesla_emp_status = pd.get_dummies(df_tesla['emp_status'])


In [ ]:
## Can't use - not getting accurate optimal # of clusters, just going to use TF-IDF

# Convert tfidf to dataframe

# Code source
# https://stackoverflow.com/questions/36967666/transform-scipy-sparse-csr-to-pandas

#df_tfidf_google = pd.DataFrame(train_tfidf_vectors_google.toarray())
#df_tfidf_tesla = pd.DataFrame(train_tfidf_vectors_tesla.toarray())

In [ ]:
## Can't use - not getting accurate optimal # of clusters, just going to use TF-IDF

# Concatenate features to one dataframe

#df_med_google = pd.concat([df_google.rating, df_google_emp_status, df_tfidf_google], axis=1, ignore_index=True) 
#df_med_google = df_med_google.rename(columns= {0: 'rating', 1: 'current', 2: 'former' })


#df_med_tesla = pd.concat([df_tesla.rating, df_tesla_emp_status, df_tfidf_tesla], axis=1, ignore_index=True) 
#df_med_tesla = df_med_tesla.rename(columns= {0: 'rating', 1: 'current', 2: 'former'})



In [ ]:
# Convert to array for lsa
google_array = np.asarray(df_med_google)
tesla_array = np.asarray(df_med_tesla)

In [ ]:
google_array

In [ ]:
# LSA feature reduction - going with 100 components - google
# Using feature reduction because pyclustering ran too long

#Code source:
# https://github.com/chrisjmccormick/LSA_Classification/blob/master/inspect_LSA.py
# https://towardsdatascience.com/2-latent-methods-for-dimension-reduction-and-topic-modeling-20ff6d7d547

svd_google = TruncatedSVD(400)
lsa_google = make_pipeline(svd_google, Normalizer(copy=False))
x_train_lsa_google = lsa_google.fit_transform(train_tfidf_vectors_google)
print('LSA output shape:', x_train_lsa_google.shape)
explained_variance_google = svd_google.explained_variance_ratio_.sum()
print("Sum of explained variance ratio: %d%%" % (int(explained_variance_google * 100)))

In [ ]:
# LSA feature reduction - going with 100 components - tesla

svd_tesla = TruncatedSVD(400)
lsa_tesla = make_pipeline(svd_tesla, Normalizer(copy=False))
x_train_lsa_tesla = lsa_tesla.fit_transform(train_tfidf_vectors_tesla)
print('LSA output shape:', x_train_lsa_tesla.shape)
explained_variance_tesla = svd_tesla.explained_variance_ratio_.sum()
print("Sum of explained variance ratio: %d%%" % (int(explained_variance_tesla * 100)))

In [ ]:
# google - pyclustering silhouette - min clusters 2, max clusters 100

# Code source
# https://codedocs.xyz/annoviko/pyclustering/classpyclustering_1_1cluster_1_1silhouette_1_1silhouette__ksearch.html

search_instance = silhouette_ksearch(x_train_lsa_google,2,100, algorithm=silhouette_ksearch_type.KMEDOIDS).process()
amount = search_instance.get_amount()
scores = search_instance.get_scores()
print("Scores: '%s'" % str(scores))

In [ ]:
# google - Get index of max score - identify optimal # of clusters

# Code source
# https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary

max(scores.items(), key=operator.itemgetter(1))[0]

In [ ]:
# tesla - pyclustering silhouette - min clusters 2, max clusters 50

search_instance = silhouette_ksearch(x_train_lsa_tesla,2,30, algorithm=silhouette_ksearch_type.KMEDOIDS).process()
amount = search_instance.get_amount()
scores = search_instance.get_scores()
print("Scores: '%s'" % str(scores))

In [ ]:
# tesla - Get index of max score - identify optimal # of clusters

max(scores.items(), key=operator.itemgetter(1))[0]

In [ ]:
# tolist needed for k-medoids
google_list = df_med_google.values.tolist()
tesla_list = df_med_tesla.values.tolist()

In [ ]:
# google - Kmedoids - with optimal clusters

# Code source
# https://github.com/annoviko/pyclustering/blob/master/pyclustering/cluster/ema.py
# https://github.com/annoviko/pyclustering/issues/366
# https://codedocs.xyz/annoviko/pyclustering/classpyclustering_1_1cluster_1_1kmedoids_1_1kmedoids.html
# https://github.com/letiantian/kmedoids

# set random initial medoids
initial_medoids_google = list(range(0,17))

# create instance of K-Medoids algorithm
kmedoids_instance_google = kmedoids(google_list, initial_medoids_google)

# run cluster analysis and obtain results
kmedoids_instance_google.process();
clusters_google = kmedoids_instance_google.get_clusters()
medoids_google = kmedoids_instance_google.get_medoids();

print("Amount of clusters - Google:", len(clusters_google));
for cluster in clusters_google:
    print("Cluster length:", len(cluster));

In [ ]:
# tesla - Kmedoids - with optimal clusters 

# set random initial medoids
initial_medoids_tesla = list(range(0,17))

# create instance of K-Medoids algorithm
kmedoids_instance_tesla = kmedoids(tesla_list, initial_medoids_tesla)

# run cluster analysis and obtain results
kmedoids_instance_tesla.process();
clusters_tesla = kmedoids_instance_tesla.get_clusters()
medoids_tesla = kmedoids_instance_tesla.get_medoids();

print("Amount of clusters - Tesla:", len(clusters_tesla));
for cluster in clusters_tesla:
    print("Cluster length:", len(cluster));

In [ ]:
# google - Create a file with cluster and original index of cluster

# Code from Dr. George

cluster_number_google = []
indices_google = []
for i, c in enumerate(clusters_google):
    cluster_number_google.extend([i] * len(c))
    indices_google.extend(c)

df_google_cluster = pd.DataFrame({'cluster': cluster_number_google, 'index': indices_google})

In [ ]:
# tesla - Create a file with cluster and original index of cluster

# Code from Dr. George

cluster_number_tesla = []
indices_tesla = []
for i, c in enumerate(clusters_tesla):
    cluster_number_tesla.extend([i] * len(c))
    indices_tesla.extend(c)

df_tesla_cluster = pd.DataFrame({'cluster': cluster_number_tesla, 'index': indices_tesla})

In [ ]:
df_google_cluster.head()

In [ ]:
# Line up indexes and add cluster to original dataframes

df_google_cluster.set_index('index', inplace=True)
df_tesla_cluster.set_index('index', inplace=True)

df_google_cluster.sort_index(inplace=True)
df_tesla_cluster.sort_index(inplace=True)

df_med_google['cluster'] = df_google_cluster['cluster']
df_med_tesla['cluster'] = df_tesla_cluster['cluster']

In [ ]:
df_med_google.head(2)

In [ ]:
# Group by cluster and capture mean of each variable

#Code source
# https://stackoverflow.com/questions/30328646/python-pandas-group-by-in-group-by-and-average

df_grouped_google = df_med_google.groupby(['cluster']).mean()
df_grouped_tesla = df_med_tesla.groupby(['cluster']).mean()

In [ ]:
# No longer needed - Could not add additional features to K-Medoids
# Drop some columns not needed for cluster analysis

# df_grouped_google.drop(['rating','SA','current','former'], inplace=True, axis=1)
# df_grouped_tesla.drop(['rating','SA','current','former'], inplace=True, axis=1)

In [ ]:
# Going to re-use countvectorizer, grabbing the clean reviews used earlier

df_google_cluster_words = pd.DataFrame(cln_review_google)
df_tesla_cluster_words = pd.DataFrame(cln_review_tesla)

In [ ]:
# Lining up the assigned cluster with each review

df_google_cluster_words['cluster'] = df_google_cluster['cluster']
df_tesla_cluster_words['cluster'] = df_tesla_cluster['cluster']

In [ ]:
df_google_cluster_words.head()

In [ ]:
# Which google clusters have the highest total average tfidf
df_google_summed = df_grouped_google.sum(axis=1)
df_google_summed.sort_values(ascending=False)

In [ ]:
# Which tesla clusters have the highest total average tfidf
df_tesla_summed = df_grouped_tesla.sum(axis=1)
df_tesla_summed.sort_values(ascending=False)

In [ ]:
# Google top 3 clusters - filter records for top clusters into their own file
first_google_cluster = df_google_cluster_words.loc[df_google_cluster_words['cluster'] == 4]
second_google_cluster = df_google_cluster_words.loc[df_google_cluster_words['cluster'] == 1]
third_google_cluster = df_google_cluster_words.loc[df_google_cluster_words['cluster'] == 16]

In [ ]:
# Tesla top 3 clusters - filter records for top clusters into their own file
first_tesla_cluster = df_tesla_cluster_words.loc[df_tesla_cluster_words['cluster'] == 10]
second_tesla_cluster = df_tesla_cluster_words.loc[df_tesla_cluster_words['cluster'] == 11]
third_tesla_cluster = df_tesla_cluster_words.loc[df_tesla_cluster_words['cluster'] == 2]

In [ ]:
# Drop the cluster variable to count words
first_google_cluster.drop(columns = 'cluster', inplace=True, axis=1)
second_google_cluster.drop(columns = 'cluster', inplace=True, axis=1)
third_google_cluster.drop(columns = 'cluster', inplace=True, axis=1)

first_tesla_cluster.drop(columns = 'cluster', inplace=True, axis=1)
second_tesla_cluster.drop(columns = 'cluster', inplace=True, axis=1)
third_tesla_cluster.drop(columns = 'cluster', inplace=True, axis=1)

In [ ]:
# Make lists for countvectorizer
first_google_cluster_list = first_google_cluster.values.tolist()
second_google_cluster_list = second_google_cluster.values.tolist()
third_google_cluster_list = third_google_cluster.values.tolist()

first_tesla_cluster_list = first_tesla_cluster.values.tolist()
second_tesla_cluster_list = second_tesla_cluster.values.tolist()
third_tesla_cluster_list = third_tesla_cluster.values.tolist()

In [ ]:
# Google cluster 4
# Report out top 10 for 1,2,3, and 4 grams

ngrams_google, ngram_counts_google = {}, {}

first_google_cluster = [str(first_google_cluster_list)]

print ('Cluster 4')

for n in [1, 2, 3, 4]:
    print('top 10', str(n) + '-grams:\n')
    ngrams_google[n], ngram_counts_google[n] = get_top_grams_google(first_google_cluster, n=n)
    print('\n')

In [ ]:
# Google cluster 1
# Report out top 10 for 1,2,3, and 4 grams

ngrams_google, ngram_counts_google = {}, {}

print ('Cluster 1')

second_google_cluster = [str(second_google_cluster_list)]

for n in [1, 2, 3, 4]:
    print('top 10', str(n) + '-grams:\n')
    ngrams_google[n], ngram_counts_google[n] = get_top_grams_google(second_google_cluster, n=n)
    print('\n')

In [ ]:
# Google cluster 16
# Report out top 10 for 1,2,3, and 4 grams

ngrams_google, ngram_counts_google = {}, {}

print ('Cluster 16')

third_google_cluster = [str(third_google_cluster_list)]

for n in [1, 2, 3, 4]:
    print('top 10', str(n) + '-grams:\n')
    ngrams_google[n], ngram_counts_google[n] = get_top_grams_google(third_google_cluster, n=n)
    print('\n')

In [ ]:
# Tesla cluster 10
# Report out top 10 for 1,2,3, and 4 grams

ngrams_tesla, ngram_counts_tesla = {}, {}

first_tesla_cluster = [str(first_tesla_cluster_list)]

print ('Cluster 10')

for n in [1, 2, 3, 4]:
    print('top 10', str(n) + '-grams:\n')
    ngrams_tesla[n], ngram_counts_tesla[n] = get_top_grams_tesla(first_tesla_cluster, n=n)
    print('\n')

In [ ]:
# Tesla cluster 11
# Report out top 10 for 1,2,3, and 4 grams

ngrams_tesla, ngram_counts_tesla = {}, {}

second_tesla_cluster = [str(second_tesla_cluster_list)]

print ('Cluster 11')

for n in [1, 2, 3, 4]:
    print('top 10', str(n) + '-grams:\n')
    ngrams_tesla[n], ngram_counts_tesla[n] = get_top_grams_tesla(second_tesla_cluster, n=n)
    print('\n')

In [ ]:
# Tesla cluster 2
# Report out top 10 for 1,2,3, and 4 grams

ngrams_tesla, ngram_counts_tesla = {}, {}

third_tesla_cluster = [str(third_tesla_cluster_list)]

print ('Cluster 2')

for n in [1, 2, 3, 4]:
    print('top 10', str(n) + '-grams:\n')
    ngrams_tesla[n], ngram_counts_tesla[n] = get_top_grams_tesla(third_tesla_cluster, n=n)
    print('\n')

## LDA/NMF

In [ ]:
# Code Source
# https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_
# nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
# Fit the NMF model 1 - google

n_components = 10
n_top_words = 10

nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(train_tfidf_vectors_google)

tfidf_feature_names = tfidf_vectorizer_google.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

In [ ]:
# Fit the NMF model 2 - google
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(train_tfidf_vectors_google)

tfidf_feature_names = tfidf_vectorizer_google.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

In [ ]:
# Fit the LDA model 1 - google
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(train_tfidf_vectors_google)

tf_feature_names = tfidf_vectorizer_google.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
# Fit the NMF model 1 - tesla

n_components = 10
n_top_words = 10

nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(train_tfidf_vectors_tesla)

tfidf_feature_names = tfidf_vectorizer_tesla.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

In [ ]:
# Fit the NMF model 2 - tesla
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(train_tfidf_vectors_tesla)

tfidf_feature_names = tfidf_vectorizer_tesla.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

In [ ]:
# Fit the LDA model 1 - tesla
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(train_tfidf_vectors_tesla)

tf_feature_names = tfidf_vectorizer_tesla.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)